In [ ]:
date = "20250519"

In [ ]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import gc
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import confusion_matrix,f1_score,classification_report,accuracy_score
from sklearn.metrics import roc_auc_score 
 
from scipy.stats import norm
import matplotlib.pyplot as plt 
# import seaborn as sns
import time
import os
import openpyxl

import joblib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv1D,Input
from tensorflow.keras.layers import MaxPooling1D,UpSampling1D,Lambda,Reshape
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.models import Model

from tensorflow.keras.losses import mse
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.utils import class_weight

# for now()
import datetime
# for timezone()
import pytz
# using now() to get current time
current_time = datetime.datetime.now(pytz.timezone('America/New_York'))
# printing current time in india
print("The current time is :", current_time)

pd.options.mode.chained_assignment = None

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
# _celline = ['A549','GM12878','H1.hESC','HeLa.S3','HepG2','HT1080','HUVEC','IMR.90','K562','MCF.7','NCI.H460','NHEK','SK.MEL.5','SK.N.DZ','SK.N.SH']
_celline = ['A549','GM12878','HeLa.S3']

K = 6
miss = [2]

threshold = "0"


# # input path
path_input = "../data/mer%s/"%(K)
# output path
path_output = "../results/Train_one_test_all/mer%s_%s/"%(K,date)

path_models = path_output+"models/"

os.makedirs(path_output, exist_ok=True)
os.makedirs(path_models,exist_ok=True)


CV = 5
max_len = 5000
segment = [(200,5000)]

_model_type = ["RF"]
encoding = "rawcounts"

# <font color=blue> Load data</font>
##  <font color=blue> load cell_line, kmer counts table, generate two class training set, mean as threshold. </font>

In [ ]:
def assign_class(row, Celline):
    if row[Celline] < 0:
        return 0
    else:
        return 1


def processdata(df):
    x_train_ = df.iloc[:,0:-5]
    y_train_ = df.iloc[:,-1]
    train_rci_ = df.iloc[:,-5:]
    
    return x_train_,y_train_,train_rci_


from collections import defaultdict
from math import sqrt
def evaluate_matrix(cm):
    idx = [0,1]
    columns = ['Sn','Sp','Preci','MCC','F1-score','OA']
    eva = pd.DataFrame(index=idx,columns=columns)
    
    oa = np.diag(cm).sum()/cm.sum()*100
    
    for i in range(len(cm)):
        fp = cm.sum(axis=0)[i]-cm[i][i] # sum of column
        fn = cm.sum(axis=1)[i]-cm[i][i] # sum of row
        tp = cm[i][i]
        tn = cm.sum()-(fp+fn+tp)
        if tp == 0:
            sensitivity = 0
            precision = 0
        else:
            sensitivity = tp / (tp+fn)
            precision = tp / (tp+fp)
            
        specificity = tn / (tn+fp)
               
        if precision*sensitivity==0:
            f1_score=0
        else:
            f1_score = 2 * precision * sensitivity / (precision + sensitivity)
        if tp*tn-fp*fn == 0:
            mcc = 0
        else:
            mcc = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
        if i==0:
            eva.loc[i] = [sensitivity,specificity,precision,mcc,f1_score,oa]
        else:
            eva.loc[i] = [sensitivity,specificity,precision,mcc,f1_score,0]
    
    return oa , eva

def evaluateModel(model_types,model,x_val,y_val,dff_val):
    if model_types == "RF":
        y_pred_prob = model.predict_proba(x_val)[:, 1] 
    else:
        y_pred_prob = model.predict(x_val).ravel() 
    auc = roc_auc_score(y_val, y_pred_prob)
    
    y_pred = (model.predict(x_val) > 0.5).astype("int32")    
    accuracy = accuracy_score(y_val, y_pred)
    
    cm = confusion_matrix(y_val, y_pred)
    oa, eva = evaluate_matrix(cm)
    eva.loc[0,"AUC"] = auc
    dff_val = pd.concat([dff_val,pd.DataFrame(y_pred,columns=["pred"]),pd.DataFrame(y_pred_prob,columns=["prob"])],axis=1)
    

    return oa, cm, eva,dff_val,auc 

In [ ]:
def train_model(Cell_train):
    
    # COUNT Running time
    start = time.time()

    
    for j in range(0,2):
    
        # training dataset split by genes, 5-fold CV
        kf = KFold(n_splits=CV,random_state=(j+1)*42, shuffle=True)
        i=0
        for train_index, val_index in kf.split(genes):
            i += 1
            print("No.{} fold".format(j*5+i))      

            for (m,n) in segment:
                # print()
                # print("Training on {}~{}".format(m,n))

                train_genes = genes.iloc[train_index].gene_id
                val_genes = genes.iloc[val_index].gene_id
                df_train = _df_train.copy()   ## here
#                 df_test = _df_test.copy()

                train = df_train.iloc[train_index]
                train = train.reset_index(drop=True)                
                x_train,y_train,train_rci = processdata(train)                

                count = Counter(y_train)
                ratio = count[0]/count[1]
                print("class 0 : class 1 = %.3f."%(ratio))
                smt = SMOTE(sampling_strategy='minority', random_state=42)
                x_train, y_train = smt.fit_resample(x_train, y_train)
                print("After over-sampling",(Counter(y_train)))          
                

                val = df_train.iloc[val_index].reset_index(drop=True)
                x_val,y_val,val_rci = processdata(val)
                
                x_train = x_train.to_numpy()
                x_val = x_val.to_numpy()

                for model_type in _model_type:
                    test_set_type = "lncATLAS"

                    print()
                    ### load model
                    print("\nTraining on %s %smer%smismatch %s %s %s %s. \n"%(Cell_train,K,Q,encoding, model_type,m,n))
                    print()
                    if model_type == "1DCNN":
                        k = x_train_ae.shape[1] 
                        model = build_CNN(x_train_ae.shape[1])            
                        x_train_cnn = x_train_ae.reshape(len(x_train_ae),k,1)
                        x_val0_cnn = x_val_ae.reshape(len(x_val_ae),k,1)

                        model,score = trainModel(ae,j*5+i,m,n,model,Cell_train,model_type,x_train_cnn,x_val0_cnn,y_train_cnn, y_val_cnn)

                        val0_eva,val0_len_acc,val0_report,val0_predit = evaluateModel(Cell_train,test_set_type,model_type,model,x_val0_cnn,y_val_cnn,val0_rci)

                        
                    elif model_type == "MLP":
                        model = build_MLP(x_train_ae.shape[1])
                        model,score = trainModel(ae,j*5+i,m,n,model,Cell_train,model_type,x_train_ae,x_val_ae,y_train_cnn, y_val_cnn)

                        val0_eva,val0_len_acc,val0_report,val0_predit = evaluateModel(Cell_train,test_set_type,model_type,model,x_val_ae,y_val_cnn,val0_rci)
                        # val_std_eva,val_std_len_acc,val_std_report,val_std_predict = evaluateModel(model,x_val_std,y_val_std,valstd_rci)

                        # test_std_eva,test_std_len_acc,test_std_report,test_std_predict = evaluateModel(model,x_test_std,y_test_std,teststd_rci)

                    elif model_type == "RF":
                        model = RandomForestClassifier(n_estimators=100)
                        model.fit(x_train,y_train)
                        joblib.dump(model,path_models+'/%s_%s_%s_%s_%s_%smer%smiss_fold%s.pkl'%(encoding,Cell_train,model_type,m,n,K,Q,j*5+i))
                        y_predit0 = model.predict(x_val)
                        score = accuracy_score(y_val,y_predit0)
                        print("RF accuracy:",score)
                        print()
                        oa_val, cm_val, eva_val, val0_predict,auc_val = evaluateModel(model_type, model,x_val,y_val,val_rci)
                    
                    print(f'model {model_type} val accu {oa_val:.3f}, auc {auc_val:.3f}\nval cm {cm_val}')
                    eva_val0_file =path_output + "10fold_train_%s_val_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_val0.xlsx"%(Cell_train,Cell_train,m,n,encoding,threshold,model_type,K,Q)
                    predict_file_val = path_output + "10fold_train_%s_val_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_val.xlsx"%(Cell_train,Cell_train,m,n,encoding,threshold,model_type,K,Q)

                    writer_eva_val = ExcelWriter(eva_val0_file, engine='openpyxl',mode='a')
                    writer_predict_val = ExcelWriter(predict_file_val, engine='openpyxl',mode='a')

                    conf_matrix_df = pd.DataFrame(cm_val, columns=['Pred_0', 'Pred_1'], index=['True_0', 'True_1'])    
                    eva_val.to_excel(writer_eva_val,sheet_name="eva{}".format(j*5+i))
                    conf_matrix_df.to_excel(writer_eva_val,sheet_name="CM{}".format(j*5+i))                    
                    val0_predict.to_excel(writer_predict_val,sheet_name="predict_0_{}".format(j*5+i))
                    writer_eva_val.close()
                    writer_predict_val.close()                                        
                    
                    for Cell_test in _celline:
                        

                        print("\nfold %s, train %s, test %s"%(j*5+i,Cell_train,Cell_test))
                        _df_test = data_test.copy()                        
                        _df_test[Cell_test] = _test[Cell_test]                        
                        _df_test = _df_test.dropna(axis=0).reset_index(drop=True)
                        _df_test["Class"] = _df_test.apply(lambda row:assign_class(row,Cell_test),axis=1)

                        x_test,y_test,test_rci = processdata(_df_test)
                        x_test = x_test.to_numpy()
                        oa_test, cm_test,eva_test,test0_predict,auc_test = evaluateModel(model_type,model,x_test,y_test,test_rci)
                        print(f'model {model_type} test accu {oa_test:.3f}, auc {auc_test:.3f}\ntest cm {cm_test}') 
                        

                        eva_test0_file = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                        predict_file_test = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_test.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)

                        writer_eva_test = ExcelWriter(eva_test0_file, engine='openpyxl',mode='a')
                        writer_predict_test = ExcelWriter(predict_file_test, engine='openpyxl',mode='a') 

                        conf_matrix_df = pd.DataFrame(cm_test, columns=['Pred_0', 'Pred_1'], index=['True_0', 'True_1'])
                        eva_test.to_excel(writer_eva_test,sheet_name="eva{}".format(j*5+i))
                        conf_matrix_df.to_excel(writer_eva_test,sheet_name="CM{}".format(j*5+i))
                        test0_predict.to_excel(writer_predict_test,sheet_name="predict_0_{}".format(j*5+i))                  

                        writer_eva_test.close()
                        writer_predict_test.close()
                        
                        
                    del model  # Explicitly delete the model
                    gc.collect()  # Run garbage collection


    end = time.time()
    print("Running time {} seconds".format(round(end-start,3)))

## <font color="blue"> only validation </font>

In [ ]:
for Q in miss:
    _train = pd.read_csv(path_input+encoding+"_noncoding_train_%smer%smiss.csv"%(K,Q),index_col=0)
    data_train = _train.iloc[:,0:-18].div(_train.iloc[:,0:-18].sum(axis=1), axis=0)
    data_train[["transcript_id","gene_id","length"]] = _train[["transcript_id","gene_id","length"]]

    _test = pd.read_csv(path_input+encoding+"_noncoding_test_%smer%smiss.csv"%(K,Q),index_col=0)
    data_test = _test.iloc[:,0:-18].div(_test.iloc[:,0:-18].sum(axis=1), axis=0)
    data_test[["transcript_id","gene_id","length"]] = _test[["transcript_id","gene_id","length"]]


    #Create output files
    wb = openpyxl.Workbook()

    for Cell_train in _celline:
        for Cell_test in _celline:
            for model_type in _model_type:
                for (m,n) in segment:
                    
                    eva_val0_file = path_output + "10fold_train_%s_val_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_val0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    predict_file_val = path_output + "10fold_train_%s_val_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_val.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    eva_test0_file = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    predict_file_test = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_test.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    wb.save(eva_val0_file)                                         
                    wb.save(predict_file_val)
                    wb.save(eva_test0_file)
                    wb.save(predict_file_test)

    for Cell_train in _celline:
        _df_train = data_train.copy()
        _df_train[Cell_train] = _train[Cell_train]
        _df_train = _df_train.dropna(axis=0)
        _df_train = _df_train.sort_values(by="length",ascending=False).reset_index(drop=True)
        _df_train["Class"] = _df_train.apply(lambda row:assign_class(row,Cell_train),axis=1)


        genes = pd.DataFrame(_df_train.gene_id.unique(),columns=["gene_id"])

        print()
        print("\n training on %s %smer%smismatch ."%(Cell_train,K,Q ))

        train_model(Cell_train)


In [ ]:
for Q in miss:
    for model_type in _model_type:
        for (m,n) in segment:
            for Cell_train in _celline:
                for Cell_test in _celline:
                    
                    eva_val0_file = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_val0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    eva_test0_file = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    for infile in [eva_test0_file]:
                        print(infile)
                        writer = ExcelWriter(infile, engine='openpyxl',mode='a',if_sheet_exists='replace')
                        xl = pd.ExcelFile(infile)
                        eva = []

                        for i in range(1,11):
                            eva.append(xl.parse('eva'+str(i),index_col=0))

                        df_eva_avg =  pd.concat(eva).groupby(level=0).mean()

                        df_eva_avg.to_excel(writer,sheet_name='eva_avg')

                        writer.close()

In [ ]:
current_time = datetime.datetime.now(pytz.timezone('America/New_York'))
# printing current time in india
print("The current time is :", current_time)